In [1]:
import pymongo as mongo
import pandas as pd
from pandas_profiling import ProfileReport

In [2]:
# connect to database

client = mongo.MongoClient("mongodb://localhost:27017")


# Select the database and collection
db = client["crm"]

crm_activity = db["crm_activity"]
crm_user = db["crm_user"]
crm_pipeline = db["crm_pipeline"]
crm_organization = db["crm_organization"]
crm_deal=db["crm_deal"]
crm_customer = db["crm_customer"]
crm_activity_type = db["crm_activity_type"]

# create a dataframe from the collection
df_activity = pd.DataFrame(list(crm_activity.find()))
df_user = pd.DataFrame(list(crm_user.find()))
df_pipeline = pd.DataFrame(list(crm_pipeline.find()))
df_organization = pd.DataFrame(list(crm_organization.find()))
df_deal = pd.DataFrame(list(crm_deal.find()))
df_customer = pd.DataFrame(list(crm_customer.find()))
df_activity_type = pd.DataFrame(list(crm_activity_type.find()))

df_user.head(5)

,_id,id,crmId,crmIsAdmin,crmName,crmDefaultCurrency,crmTimezoneOffset,crmEmail,crmPhone,crmLastLogin,crmCreateDate,createDate,crmIsActive,isDeleted,companyId
0,64902f167cc8b1037a62bdd7,1,3204921,0,Amanda Partosch,EUR,+02:00,amanda@casavi.de,None,2017-11-27 09:24:27,None,2022-01-27 14:13:34,1,0,5
1,64902f167cc8b1037a62bdd8,2,9033176,0,Anja Stolz,EUR,+02:00,anja.stolz@casavi.de,None,2019-09-09 12:04:28,None,2022-01-27 14:13:34,1,0,5
2,64902f167cc8b1037a62bdd9,3,11406786,0,Anna Gassmann,EUR,+02:00,anna.gassmann@casavi.de,None,2020-11-25 15:16:17,None,2022-01-27 14:13:34,1,0,5
3,64902f167cc8b1037a62bdda,4,12654440,0,Arne Leppelsack,EUR,+02:00,arne.leppelsack@casavi.de,None,2021-10-14 06:47:40,None,2022-01-27 14:13:34,1,0,5
4,64902f167cc8b1037a62bddb,5,11353976,0,Benjamin Grotepass,EUR,+02:00,benjamin.grotepass@casavi.de,None,2022-05-13 06:42:47,None,2022-01-27 14:13:34,1,0,5


In [3]:
#Merge the datasets
# Merge the datasets
#df = pd.merge(df_activity, df_user, on='crmUserId', suffixes=('_activity', '_user'))
df = pd.merge(df_activity, df_user, left_on='crmUserId', right_on='crmId', suffixes=('_activity', '_user'))
df = pd.merge(df, df_pipeline, left_on='crmId_activity', right_on='crmId', suffixes=('', '_pipeline'))
df = pd.merge(df, df_organization, left_on='crmId_activity', right_on='crmId', suffixes=('', '_organization'))
df = pd.merge(df, df_deal, left_on='crmDealId', right_on='crmId', suffixes=('', '_deal'))
df = pd.merge(df, df_customer, left_on='crmCustomerId', right_on='crmId', suffixes=('', '_customer'))
df = pd.merge(df, df_activity_type, left_on='crmType', right_on='key_string', suffixes=('', '_activity_type'))

df.head(5)  # This will display the first few rows of the merged DataFrame

,_id_activity,id_activity,crmId_activity,crmUserId,crmDealId,crmLeadId,crmType,crmAddTime,crmUpdateTime,crmDueDate,...,crmZipCode_customer,crmPhoneNumber,_id_activity_type,id_activity_type,key_string,name,staticActivityTypeId,companyId_activity_type,createDate_activity_type,isDeleted_activity_type
0,64902df27cc8b1037a57f1e6,10472,11,686380.0,9.0,None,deadline,2015-03-04 09:01:14,2015-10-12 10:16:31,2015-10-12,...,NaN,+49 7352 93205,64902ec17cc8b1037a5ec862,4,deadline,Deadline,4.0,11,2022-05-03 09:36:42,0
1,64902df27cc8b1037a57f1e6,10472,11,686380.0,9.0,None,deadline,2015-03-04 09:01:14,2015-10-12 10:16:31,2015-10-12,...,NaN,+49 7352 93205,64902ec17cc8b1037a5ec86b,13,deadline,Frist,NaN,9,2022-05-03 09:37:29,0
2,64902df27cc8b1037a57f1e6,10472,11,686380.0,9.0,None,deadline,2015-03-04 09:01:14,2015-10-12 10:16:31,2015-10-12,...,NaN,+49 7352 93205,64902ec17cc8b1037a5ec87f,33,deadline,Frist,3.0,12,2022-05-04 18:40:43,0
3,64902df27cc8b1037a57f1e6,10472,11,686380.0,9.0,None,deadline,2015-03-04 09:01:14,2015-10-12 10:16:31,2015-10-12,...,NaN,+49 7352 93205,64902ec17cc8b1037a5ec888,42,deadline,Deadline,1.0,13,2022-05-05 13:40:38,0
4,64902df27cc8b1037a57f1e6,10472,11,686380.0,9.0,None,deadline,2015-03-04 09:01:14,2015-10-12 10:16:31,2015-10-12,...,NaN,+49 7352 93205,64902ec17cc8b1037a5ec88e,48,deadline,Deadline,NaN,14,2022-05-06 04:06:06,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15740175 entries, 0 to 15740174
Data columns (total 96 columns):
 #   Column                     Dtype         
---  ------                     -----         
 0   _id_activity               object        
 1   id_activity                int64         
 2   crmId_activity             int64         
 3   crmUserId                  float64       
 4   crmDealId                  float64       
 5   crmLeadId                  object        
 6   crmType                    object        
 7   crmAddTime                 object        
 8   crmUpdateTime              object        
 9   crmDueDate                 datetime64[ns]
 10  crmSubject                 object        
 11  crmDone                    object        
 12  isDeleted_activity         object        
 13  createDate_activity        object        
 14  companyId_activity         object        
 15  crmDueTime                 object        
 16  crmDuration                object 

In [5]:
df.describe()

,id_activity,crmId_activity,crmUserId,crmDealId,id_user,crmId_user,crmIsAdmin,crmIsActive,isDeleted_user,companyId_user,...,crmCity_customer,crmStreet_customer,isDeleted_customer,companyId_customer,crmOrgId_customer,crmZipCode_customer,id_activity_type,staticActivityTypeId,companyId_activity_type,isDeleted_activity_type
count,1.574018e+07,1.574018e+07,1.574018e+07,1.574018e+07,1.574018e+07,1.574018e+07,15740175.0,15740175.0,15740175.0,1.574018e+07,...,0.0,0.0,1.574018e+07,1.574018e+07,1.359994e+07,0.0,1.574018e+07,1.268138e+07,1.574018e+07,15740175.0
mean,8.058170e+05,8.242705e+00,1.123143e+07,1.149271e+01,3.921987e+06,1.123143e+07,0.0,1.0,0.0,8.504028e+00,...,NaN,NaN,4.757253e-04,1.106471e+01,8.326435e+02,NaN,6.282177e+01,1.296633e+00,1.700008e+01,0.0
std,5.840951e+06,4.793942e+00,3.326152e+06,3.990577e+01,6.388749e+06,3.326152e+06,0.0,0.0,0.0,6.733963e+00,...,NaN,NaN,2.180594e-02,8.265200e+00,3.832680e+03,NaN,3.474287e+01,6.971805e-01,1.034075e+01,0.0
min,1.003600e+04,1.000000e+00,5.196600e+05,1.000000e+00,3.900000e+01,5.196600e+05,0.0,1.0,0.0,2.000000e+00,...,NaN,NaN,0.000000e+00,2.000000e+00,1.000000e+00,NaN,1.000000e+00,1.000000e+00,4.000000e+00,0.0
25%,1.986120e+05,5.000000e+00,1.218294e+07,5.000000e+00,1.130000e+02,1.218294e+07,0.0,1.0,0.0,3.000000e+00,...,NaN,NaN,0.000000e+00,5.000000e+00,5.000000e+00,NaN,3.900000e+01,1.000000e+00,1.100000e+01,0.0
50%,2.285530e+05,7.000000e+00,1.219530e+07,7.000000e+00,1.230000e+02,1.219530e+07,0.0,1.0,0.0,5.000000e+00,...,NaN,NaN,0.000000e+00,9.000000e+00,8.000000e+00,NaN,6.900000e+01,1.000000e+00,1.400000e+01,0.0
75%,3.005370e+05,9.000000e+00,1.251282e+07,1.100000e+01,1.429778e+07,1.251282e+07,0.0,1.0,0.0,1.200000e+01,...,NaN,NaN,0.000000e+00,1.400000e+01,6.200000e+01,NaN,8.800000e+01,1.000000e+00,1.900000e+01,0.0
max,6.090265e+07,2.000000e+01,1.349794e+07,3.171000e+03,1.516801e+07,1.349794e+07,0.0,1.0,0.0,4.200000e+01,...,NaN,NaN,1.000000e+00,4.200000e+01,4.215400e+04,NaN,1.150000e+02,4.000000e+00,4.200000e+01,0.0


In [6]:
df.shape

(15740175, 96)

In [8]:
Profile = ProfileReport(df, title = "CRM data reporting")

c:\Users\nakhu\anaconda3\envs\tf_env\lib\multiprocessing\pool.py:475: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  iterable = list(iterable)
